In [18]:
from datetime import datetime
import config, input_tables
from steps import psfsubtraction,klipphotometry
from stralog import getLogger
import os
import pkg_resources as pkg

First, we need to initialize the logger here.

In [19]:
if 'SHARED_LOG_FILE' not in os.environ:
    os.environ['SHARED_LOG_FILE'] = f'straklip_{datetime.now().strftime("%Y-%m-%d_%H%M")}.log'

getLogger('straklip', setup=True, logfile=os.environ['SHARED_LOG_FILE'],debu=False,
          configfile=pkg.resource_filename('straklip', './config/logging.yaml'))

<Logger straklip (DEBUG)>

For this tutorial we will start form the previously generated dataframes from the pipeline.
Note that in the pipe.yaml, we use unq_ids_list: [52] so the pipeline will run the reduction only on this target, to showcase the pipelien and save time and space.

In [20]:
pipe_cfg='/Users/gstrampelli/StraKLIP/Tutorial/pipeline_logs/pipe.yaml' #or where these files are
data_cfg='/Users/gstrampelli/StraKLIP/Tutorial/pipeline_logs/data.yaml'
pipe_cfg = config.configure_pipeline(pipe_cfg,pipe_cfg=pipe_cfg,data_cfg=data_cfg,dt_string=datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
data_cfg = config.configure_data(data_cfg,pipe_cfg)

2025-06-05 13:04:47 config                      :INFO     (configure_pipeline:72[pid=48146]) 
StraKLIP pipeline started at date and time: 05/06/2025 13:04:47
Pipe_cfg: /Users/gstrampelli/StraKLIP/Tutorial/pipeline_logs/pipe.yaml
Data_cfg: /Users/gstrampelli/StraKLIP/Tutorial/pipeline_logs/data.yaml

2025-06-05 13:04:47 config                      :INFO     (configure_data:153[pid=48146]) Validation of default labels and data successful!


Once the "pipe_cfg" and the "data_cfg" are configuration, we can load pre-existing dataframes.

In this case we use "skip_originals" True and "load" True to tell the pipeline tom not look for inpout catalogs, but for the dataframe already generated by the pipeline.

In [21]:
dataset = input_tables.Tables(data_cfg, pipe_cfg, skip_originals=True)
DF = config.configure_dataframe(dataset,load=True)

2025-06-05 13:04:48 config                      :WARNING  (configure_dataframe:226[pid=48146]) get_Av_dict currently only supports VEGAmag system. Please provide your own set of AVs if in a differest system as AVs : {ext: {mag_filter : value}} in the data.yaml under target
2025-06-05 13:04:48 ancillary                   :INFO     (get_Av_dict:425[pid=48146]) before dust, V =  0.0 mag(VEGA)
2025-06-05 13:04:48 ancillary                   :INFO     (get_Av_dict:426[pid=48146]) after dust, V = 1.0146 mag(VEGA)
2025-06-05 13:04:48 ancillary                   :INFO     (get_Av_dict:445[pid=48146]) Av = 1.0146 mag
2025-06-05 13:04:49 ancillary                   :INFO     (get_Av_dict:489[pid=48146]) AV=0 wfc3,uvis2,f814w 0.0 mag(VEGA)
2025-06-05 13:04:49 ancillary                   :INFO     (get_Av_dict:490[pid=48146]) AV=1 wfc3,uvis2,f814w 0.6094 mag
2025-06-05 13:04:49 ancillary                   :INFO     (get_Av_dict:489[pid=48146]) AV=0 wfc3,uvis2,f850lp 0.0 mag(VEGA)
2025-06-05 13:04:

In [22]:
DF.keys

['unq_targets',
 'crossmatch_ids',
 'mvs_targets',
 'mvs_candidates',
 'unq_candidates']

The next step wil perform PSF subtraction for each target in the "mvs_targets_df", populating both the "mvs_candidates_df" and the "unq_candidates_df, creating a "residual_tiles" folder in the "out" directory with visual summary of the subtraction and updating the tile cube in the "mvs_tiles" and "median_tiles" with the outcome of the subtraction: a.k.a. the residuals for each KLIP mode and the models.

In [23]:
psfsubtraction.run({'DF': DF, 'dataset': dataset})

2025-06-05 13:04:51 steps.psfsubtraction        :INFO     (run:237[pid=48146]) Performing KLIP PSF subtraction on tiles.
2025-06-05 13:04:51 straklip.config             :INFO     (make_paths:117[pid=48146]) "/Users/gstrampelli/StraKLIP/Tutorial/out/residual_tiles/f814w" exists, and will not be created.
2025-06-05 13:04:51 ancillary                   :INFO     (parallelization_package:969[pid=48146]) Max allowable workers 1, # of elements 1 , # of chunk 1 approx # of elemtent per chunks 1 (chunksize)


100%|██████████| 1/1 [00:00<00:00,  2.51it/s]


2025-06-05 13:09:57 straklip.config             :INFO     (make_paths:117[pid=48146]) "/Users/gstrampelli/StraKLIP/Tutorial/out/residual_tiles/f850lp" exists, and will not be created.
2025-06-05 13:09:57 ancillary                   :INFO     (parallelization_package:969[pid=48146]) Max allowable workers 1, # of elements 1 , # of chunk 1 approx # of elemtent per chunks 1 (chunksize)


100%|██████████| 1/1 [00:00<00:00, 10.05it/s]


2025-06-05 13:15:05 dataframe                   :INFO     (save_dataframes:109[pid=48146]) Saving the the following keys in ['unq_targets_df', 'crossmatch_ids_df', 'mvs_targets_df', 'mvs_candidates_df', 'unq_candidates_df'] to .csv files in /Users/gstrampelli/StraKLIP/Tutorial/out


 After generating the tiles for the residuals, we run the aperture photometry on each of them, to gather some basic information for the upcoming analysis. This step provide a basic aperture photometry and candidate identification (locating the brightest pixel in the residuals that persist across different filters and/or KLIP modes as requested).

In [24]:
klipphotometry.run({'DF': DF, 'dataset': dataset})

2025-06-05 13:15:05 steps.klipphotometry        :INFO     (update_candidate_dataframe:1184[pid=48146]) Updating the candidates dataframe
2025-06-05 13:15:05 steps.klipphotometry        :INFO     (update_companion_ZPT:885[pid=48146]) Working on the zero points for candidates
2025-06-05 13:15:05 ancillary                   :INFO     (parallelization_package:969[pid=48146]) Max allowable workers 8, # of elements 57 , # of chunk 19 approx # of elemtent per chunks 3 (chunksize)
2025-06-05 13:15:08 steps.klipphotometry        :INFO     (update_candidates:996[pid=48146]) Working on the candidates
2025-06-05 13:15:08 ancillary                   :INFO     (parallelization_package:969[pid=48146]) Max allowable workers 8, # of elements 57 , # of chunk 19 approx # of elemtent per chunks 3 (chunksize)
2025-06-05 13:15:18 steps.klipphotometry        :INFO     (update_candidate_dataframe:1203[pid=48146]) Updating the median tiles for the candidates
2025-06-05 13:15:18 steps.klipphotometry        :INF

In [25]:
DF.mvs_candidates_df

,mvs_ids,x_tile_f814w,x_tile_f850lp,y_tile_f814w,y_tile_f850lp,x_rot_f814w,x_rot_f850lp,y_rot_f814w,y_rot_f850lp,counts_f814w,...,tp_above_th_f814w,tp_above_th_f850lp,tp_above_nsigma_f814w,tp_above_nsigma_f850lp,fp_above_th_f814w,fp_above_th_f850lp,fp_above_nsigma_f814w,fp_above_nsigma_f850lp,auc_f814w,auc_f850lp
0,3,NaN,22.0,NaN,21.0,NaN,17.782,NaN,20.287,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,NaN,21.0,NaN,17.0,NaN,20.879,NaN,23.038,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8,NaN,18.0,NaN,19.0,NaN,22.212,NaN,19.676,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,NaN,16.0,NaN,22.0,NaN,22.110,NaN,16.057,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,NaN,17.0,NaN,36.0,NaN,13.176,NaN,5.221,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,12,NaN,22.0,NaN,20.0,NaN,18.348,NaN,21.127,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,16,NaN,3.0,NaN,22.0,NaN,32.617,NaN,8.433,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,23,NaN,12.0,NaN,18.0,NaN,28.183,NaN,18.977,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,24,NaN,24.0,NaN,24.0,NaN,14.442,NaN,18.946,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,40,19.0,20.0,18.0,18.0,17.764,18.201,20.026,19.127,1023.287,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
DF.unq_candidates_df


,unq_ids,mass,emass,sep,mkmode,n_f814w,n_f850lp,nsigma_f814w,nsigma_f850lp,m_f814w,...,tp_above_th_f814w,tp_above_th_f850lp,tp_above_nsigma_f814w,tp_above_nsigma_f850lp,fp_above_th_f814w,fp_above_th_f850lp,fp_above_nsigma_f814w,fp_above_nsigma_f850lp,auc_f814w,auc_f850lp
0,3,NaN,NaN,2.236,8.0,NaN,1.0,NaN,11.154746,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,NaN,NaN,3.162,9.0,NaN,1.0,NaN,9.412458,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8,NaN,NaN,2.236,7.0,NaN,1.0,NaN,9.066995,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,NaN,NaN,4.472,10.0,NaN,1.0,NaN,7.060312,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,NaN,NaN,16.279,7.0,NaN,1.0,NaN,6.978499,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,12,NaN,NaN,2.000,9.0,NaN,1.0,NaN,5.780697,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,16,NaN,NaN,17.117,35.0,NaN,1.0,NaN,5.753091,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,23,NaN,NaN,8.246,45.0,NaN,1.0,NaN,6.091083,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,24,NaN,NaN,5.657,35.0,NaN,1.0,NaN,5.624348,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,40,NaN,NaN,2.118,8.0,1.0,1.0,16.448718,11.552770,24.224,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Once the basic photometry has been performed on the candidates, we can move on on the final stage of the pipeline and perform some more